# 日々の軌道高度データを作成する(その2)

毎日0:00UT時点での軌道要素データ(軌道長半径・近点・遠点)を線形補間で作成する。0:00UT時点での実際の衛星の位置ではない。

複数の EPOCH の時間間隔が1秒以下の時には、CREATION_DATE が新しいもの1つを残して、他は削除してから、補間を行う。
(必ずしも最後のEPOCHが残るわけではない)

`json2sqlite3.py` と `satcat2sqlite3.py` で作成したデータベースを用いる。

In [1]:
import pandas as pd
import numpy as np
import os
import math
import sqlite3
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 80)

In [3]:
db_elset = 'db/elset.sqlite3'
db_satcat = 'db/satcat.sqlite3'

In [4]:
# 軌道長半径、近地点、遠地点の時間変化をプロットするための関数
def plot4(df, index, title = None, filename = None, noshow = False):
    fig = plt.figure(figsize=(10, 12))
    ax1 = fig.add_subplot(3, 1, 1)
    if title is not None: ax1.set_title(title)
    ax1.plot(index, df['SEMIMAJOR_AXIS'], color='#aaccff', marker='.', markersize=1, markeredgecolor='#5070a0')
    #ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, bymonth=1, tz=None))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Semimajor Axis [km]")
    ax1.grid(b=True, which='major', color='#bbbbbb', linestyle='-')
    ax2 = fig.add_subplot(3, 1, 2)
    ax2.plot(index, df['PERIAPSIS'], color='#aaccff', marker='.', markersize=1, markeredgecolor='#5070a0')
    #ax2.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, bymonth=1, tz=None))
    ax2.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    ax2.set_xlabel("Date")
    ax2.set_ylabel("Perigee [km]")
    ax2.grid(b=True, which='major', color='#bbbbbb', linestyle='-')
    ax3 = fig.add_subplot(3, 1, 3)
    ax3.plot(index, df['APOAPSIS'], color='#aaccff', marker='.', markersize=1, markeredgecolor='#5070a0')
    #ax3.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, bymonth=1, tz=None))
    ax3.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    ax3.set_xlabel("Date")
    ax3.set_ylabel("Apogee [km]")
    ax3.grid(b=True, which='major', color='#bbbbbb', linestyle='-')
    fig.autofmt_xdate()
    plt.tight_layout()
    if filename is not None: plt.savefig(filename)
    if not noshow:
        plt.show()
    else:
        plt.close()

In [5]:
# 軌道長半径、近地点、遠地点の日変化をプロット
def plotdot(df, set_ylim = False, title = None, filename = None, noshow = False):
    fig = plt.figure(figsize=(10, 12))
    ax1 = fig.add_subplot(3, 1, 1)
    if title is not None: ax1.set_title(title)
    ax1.plot(df['DOT_SEMIMAJOR_AXIS'], color='#aaccff', marker='.', markersize=1, markeredgecolor='#5070a0')
    #ax1.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, bymonth=1, tz=None))
    ax1.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    ax1.set_xlabel("Date")
    ax1.set_ylabel("Change of Semimajor Axis [km / day]")
    ax1.grid(b=True, which='major', color='#bbbbbb', linestyle='-')
    if set_ylim:  ax1.set_ylim(myrange(df_daily['DOT_SEMIMAJOR_AXIS']))
    ax2 = fig.add_subplot(3, 1, 2)
    ax2.plot(df['DOT_PERIAPSIS'], color='#aaccff', marker='.', markersize=1, markeredgecolor='#5070a0')
    #ax2.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, bymonth=1, tz=None))
    ax2.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    ax2.set_xlabel("Date")
    ax2.set_ylabel("Change of Perigee [km / day]")
    ax2.grid(b=True, which='major', color='#bbbbbb', linestyle='-')
    if set_ylim: ax2.set_ylim(myrange(df_daily['DOT_PERIAPSIS']))
    ax3 = fig.add_subplot(3, 1, 3)
    ax3.plot(df['DOT_APOAPSIS'], color='#aaccff', marker='.', markersize=1, markeredgecolor='#5070a0')
    #ax3.xaxis.set_major_locator(mdates.MonthLocator(bymonthday=1, bymonth=1, tz=None))
    ax3.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))
    ax3.set_xlabel("Date")
    ax3.set_ylabel("Change of Apogee [km / day]")
    ax3.grid(b=True, which='major', color='#bbbbbb', linestyle='-')
    if set_ylim: ax3.set_ylim(myrange(df_daily['DOT_APOAPSIS']))
    fig.autofmt_xdate()
    plt.tight_layout()
    if filename is not None: plt.savefig(filename)
    if not noshow:
        plt.show()
    else:
        plt.close()

In [6]:
# 軌道長半径日変化のヒストグラムをプロット
def plothist(df, title = None, filename = None, noshow = False):
    fig = plt.figure(figsize=(10, 6))
    ax1 = fig.add_subplot(1, 1, 1)
    with np.errstate(invalid='ignore'):
        ax1.hist(df_daily['DOT_SEMIMAJOR_AXIS'], bins=100, range = (-0.1,0.05))
    if title is not None: ax1.set_title(title)
    ax1.set_xlabel("Change of Semimajor Axis [km / day]")
    ax1.set_ylabel("N")
    ax1.grid(b=True, which='major', color='#bbbbbb', linestyle='-')
    ax1.minorticks_on()
    ax1.grid(b=True, which='minor', color='#bbbbbb', linestyle='-', alpha=0.2)
    plt.tight_layout()
    if filename is not None: plt.savefig(filename)
    if not noshow:
        plt.show()
    else:
        plt.close()

In [7]:
def myrange(data, percentile = 0.5, expand = 20):
    r = np.nanpercentile(data, [percentile, 100-percentile])
    r[0] = max(r[0], min(data))
    r[1] = min(r[1], max(data))
    w = (r[1] - r[0]) * expand / 100
    r += [-w, +w]
    return r

In [8]:
with sqlite3.connect(db_satcat) as conn:
    satcat = pd.read_sql_query('''
        SELECT
            *
        FROM
            satcat
        WHERE
            INCLINATION BETWEEN 80 AND 100
            AND APOGEE BETWEEN 600 AND 750
            AND PERIGEE BETWEEN 600 AND 750
            AND ( DECAY IS NULL OR DECAY >= "2015-07-01" )
            AND ( LAUNCH <= "2013-06-30" )
        ORDER BY
            NORAD_CAT_ID
    ''', conn, parse_dates = ['LAUNCH', 'DECAY'])
print(len(satcat))

720


In [9]:
count = 0
with sqlite3.connect(db_elset) as conn:
    for norad_cat_id, object_name in zip(satcat['NORAD_CAT_ID'], satcat['OBJECT_NAME']):
        print(norad_cat_id, object_name)
        # データを読み込み
        df = pd.read_sql_query("SELECT * FROM elset WHERE NORAD_CAT_ID = {}".format(norad_cat_id),
                                  conn, parse_dates = ['CREATION_DATE', 'EPOCH'])
        #print(len(df), 'records read')
        if len(df) == 0:
            print("No data")
            continue
        # EPOCH でソートする
        df = df.sort_values(['EPOCH', 'CREATION_DATE']).reset_index(drop=True)
        # 期間内のデータが含まれているかどうか確認
        if df.at[df.index[0], 'EPOCH'] > datetime(2013, 6, 30) or df.at[df.index[-1], 'EPOCH'] < datetime(2015, 7, 1):
            print("No enough data")
            continue
        # 軌道長半径・遠地点高度・近地点高度を再計算する
        df['SEMIMAJOR_AXIS'] = (398600.4418 / (df['MEAN_MOTION'] * 2 * math.pi / (24 * 3600)) ** 2) ** (1/3)
        df['APOAPSIS'] = (df['SEMIMAJOR_AXIS'] * (1 + df['ECCENTRICITY']))- 6378.135
        df['PERIAPSIS'] = (df['SEMIMAJOR_AXIS'] * (1 - df['ECCENTRICITY']))- 6378.135
        # EPOCH間隔が1秒以内のレコードを削除
        dt = -df['EPOCH'].diff(-1) / pd.Timedelta(seconds = 1)
        dt2 = df['EPOCH'].diff(1) / pd.Timedelta(seconds = 1)
        flag = (dt <= 1) | (dt2 <= 1)
        df['group'] = 0
        nlargegroups = 0
        currentgroupid = None
        first_record_in_group = None
        delta = pd.Timedelta(seconds = 1)
        for index in df.index:
            if index == 0 or (df.at[index, 'EPOCH'] - df.at[index - 1, 'EPOCH']) > delta:
                currentgroupid = index
                first_record_in_group = True
            else:
                if first_record_in_group:
                    first_record_in_group = False
                    currentgroupid = index - 1
                    nlargegroups += 1

            df.at[index, 'group'] = currentgroupid
        #print(flag.sum() - nlargegroups, 'dup records')
        df_out = df.sort_values(['group', 'CREATION_DATE']).drop_duplicates(subset='group', keep='last').sort_values(['EPOCH', 'CREATION_DATE']).drop('group', axis=1)
        #print(len(df_out), 'records remain')
        # 毎日0時の値を線形補間で算出する
        date_start = df_out.at[df_out.index[0], 'EPOCH'].ceil('D')
        date_end = df_out.at[df_out.index[-1], 'EPOCH'].floor('D')
        epoch_daily = pd.date_range(date_start, date_end, freq='D')
        df_daily = pd.DataFrame(index = epoch_daily)
        df_daily.index.name = 'DATE'
        time_in = [x.to_julian_date() for x in df_out['EPOCH']]
        time_out = [x.to_julian_date() for x in epoch_daily]
        df_daily['SEMIMAJOR_AXIS'] = np.interp(time_out, time_in, df_out['SEMIMAJOR_AXIS'])
        df_daily['PERIAPSIS'] = np.interp(time_out, time_in, df_out['PERIAPSIS'])
        df_daily['APOAPSIS'] = np.interp(time_out, time_in, df_out['APOAPSIS'])
        df_daily['DOT_SEMIMAJOR_AXIS'] = df_daily['SEMIMAJOR_AXIS'].diff()
        df_daily['DOT_PERIAPSIS'] = df_daily['PERIAPSIS'].diff()
        df_daily['DOT_APOAPSIS'] = df_daily['APOAPSIS'].diff()
        # CSVファイルに保存する
        filebase = "{}_{}".format(norad_cat_id, object_name.replace(' ', '_').replace('/', '-'))
        #print(csvfile)
        df_daily.to_csv(filebase + '.csv', columns = ['SEMIMAJOR_AXIS', 'PERIAPSIS', 'APOAPSIS', 'DOT_SEMIMAJOR_AXIS', 'DOT_PERIAPSIS', 'DOT_APOAPSIS'])
        # plot
        plot4(df, df['EPOCH'], title='{} {}'.format(norad_cat_id, object_name), filename=filebase + '.png', noshow = True)
        plotdot(df_daily, set_ylim = True, title='{} {}'.format(norad_cat_id, object_name), filename=filebase + '_dot.png', noshow = True)
        plothist(df, title='{} {}'.format(norad_cat_id, object_name), filename=filebase + '_hist.png', noshow = True)
        count += 1
print(count)

1433 DELTA 1 R/B
1983 DELTA 1 R/B
2154 DELTA 1 DEB
2985 THOR BURNER 2 R/B
4638 THORAD AGENA D DEB
4746 THORAD AGENA D DEB
4769 THORAD AGENA D DEB
4851 THORAD AGENA D DEB
4852 THORAD AGENA D DEB
5154 THORAD AGENA D DEB
5560 ASTEX 1
5694 THORAD AGENA D DEB
5842 THORAD AGENA D DEB
6180 SCOUT B-1 R/B
6217 OPS 8180 (STP RADSAT)
6218 ATLAS F BURNER 2 R/B
6284 SCOUT B-1 DEB
8826 METEOR 1-10 DEB
8951 DELTA 1 DEB
8956 DELTA 1 DEB
8958 DELTA 1 DEB
8978 DELTA 1 DEB
8980 DELTA 1 DEB
9003 DELTA 1 DEB
9320 DELTA 1 DEB
9907 SL-3 DEB
11587 SL-8 DEB
12230 DELTA 1 DEB
12269 DELTA 1 DEB
12279 DELTA 1 DEB
12559 NOAA 7 DEB
12560 NOAA 7 DEB
12755 COSMOS 1275 DEB
12756 COSMOS 1275 DEB
12779 COSMOS 1275 DEB
12786 SL-14 R/B
12988 SL-14 R/B
13272 SL-14 R/B
13479 COSMOS 1275 DEB
13482 COSMOS 1275 DEB
13553 SL-14 R/B
13809 COSMOS 1275 DEB
13826 DELTA 1 DEB
14033 SL-14 R/B
14148 SL-14 R/B
14222 SCOUT G-1 DEB
14373 SL-14 R/B
14552 SL-14 R/B
14559 DELTA 1 DEB
14700 SL-14 R/B
14781 OSCAR 11 (UoSAT 2)
14820 SL-14 R/B


35488 IRIDIUM 33 DEB
35616 IRIDIUM 33 DEB
35624 IRIDIUM 33 DEB
35678 IRIDIUM 33 DEB
35681 DEIMOS 1
35682 DUBAISAT 1
35683 DMC 2
35689 SL-24 DEB
35737 IRIDIUM 33 DEB
35739 IRIDIUM 33 DEB
35742 IRIDIUM 33 DEB
35744 IRIDIUM 33 DEB
35745 IRIDIUM 33 DEB
35749 IRIDIUM 33 DEB
35797 IRIDIUM 33 DEB
35808 IRIDIUM 33 DEB
No enough data
35809 IRIDIUM 33 DEB
35848 IRIDIUM 33 DEB
35850 IRIDIUM 33 DEB
35851 IRIDIUM 33 DEB
35853 IRIDIUM 33 DEB
35854 IRIDIUM 33 DEB
35857 IRIDIUM 33 DEB
35858 IRIDIUM 33 DEB
35862 IRIDIUM 33 DEB
35915 IRIDIUM 33 DEB
35917 IRIDIUM 33 DEB
35922 IRIDIUM 33 DEB
35925 IRIDIUM 33 DEB
35926 IRIDIUM 33 DEB
35929 IRIDIUM 33 DEB
35931 OCEANSAT 2
35932 SWISSCUBE
35933 BEESAT
35934 UWE-2
35935 ITUPSAT 1
35955 ARIANE 40 DEB
36018 IRIDIUM 33 DEB
No enough data
36019 IRIDIUM 33 DEB
36023 IRIDIUM 33 DEB
36025 IRIDIUM 33 DEB
36028 IRIDIUM 33 DEB
36037 PROBA 2
36038 SL-19 DEB
36088 SJ-11-01
36090 CZ-2C DEB
36091 CZ-2C DEB
36110 YAOGAN 7
36181 FENGYUN 1C DEB
36237 FENGYUN 1C DEB
36294 NADE